# Miami Hotel Search Engine Demo

This notebook demonstrates four different hotel search engines:
1. Generic Search Engine API
2. DuckDuckGo Search
3. Traversaal Search
4. Local Qdrant Vector Search

Each engine has its own strengths and use cases:
- Generic Search: Broad web search with customizable parameters
- DuckDuckGo: Privacy-focused search with real-time results
- Traversaal: Specialized hotel search with rich metadata
- Qdrant Local: Fast semantic search with local data

In [71]:
# Install required packages
!pip install -r ../requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [72]:
import os
import sys
import importlib
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
import json

# Add src to path
sys.path.append('..')

# Load environment variables
load_dotenv()

# Import base module first
import src.search_engines.base as base
importlib.reload(base)

# Import and reload search engine modules
import src.search_engines.generic as generic
import src.search_engines.duckduckgo as duckduckgo
import src.search_engines.traversaal as traversaal
import src.search_engines.qdrant_local as qdrant_local

# Reload all modules in reverse dependency order
importlib.reload(qdrant_local)
importlib.reload(traversaal)
importlib.reload(duckduckgo)
importlib.reload(generic)

# Import search engines after reload
from src.search_engines.generic import GenericSearchEngine
from src.search_engines.duckduckgo import DuckDuckGoSearchEngine
from src.search_engines.traversaal import TraversaalSearchEngine
from src.search_engines.qdrant_local import QdrantLocalSearchEngine

## Initialize Search Engines

Each search engine requires different initialization parameters:
- Generic Search: Path to hotel data CSV
- DuckDuckGo: No API key needed
- Traversaal: API key
- Qdrant Local: Path to hotel data CSV

In [73]:
# Initialize search engines
engines = {}

# Generic Search Engine
try:
    engines['generic'] = GenericSearchEngine(data_path="../../004-module/data/miami_hotels.csv")
    print("✓ Generic Search Engine initialized")
except Exception as e:
    print(f"✗ Generic Search Engine failed: {str(e)}")

# DuckDuckGo Search Engine
try:
    engines['duckduckgo'] = DuckDuckGoSearchEngine()
    print("✓ DuckDuckGo Search Engine initialized")
except Exception as e:
    print(f"✗ DuckDuckGo Search Engine failed: {str(e)}")

# Traversaal Search Engine
try:
    engines['traversaal'] = TraversaalSearchEngine()
    print("✓ Traversaal Search Engine initialized")
except Exception as e:
    print(f"✗ Traversaal Search Engine failed: {str(e)}")

# Qdrant Local Search Engine
try:
    engines['qdrant'] = QdrantLocalSearchEngine(data_path="../../004-module/data/miami_hotels.csv")
    print("✓ Qdrant Local Search Engine initialized")
except Exception as e:
    print(f"✗ Qdrant Local Search Engine failed: {str(e)}")

Loaded existing embeddings from storage
✓ Generic Search Engine initialized
✓ DuckDuckGo Search Engine initialized
✓ Traversaal Search Engine initialized
Using existing Qdrant collection
✓ Qdrant Local Search Engine initialized


## Search Function

This function will run a search query across all initialized engines and display the results in a formatted way.

In [74]:
def search_hotels(query: str, top_k: int = 2):
    """Search for hotels using all available engines.
    
    Args:
        query: Search query string
        top_k: Number of results to return per engine
    """
    results = {}
    
    # Run search on each engine
    for name, engine in engines.items():
        try:
            results[name] = engine.search(query, top_k=top_k)
        except Exception as e:
            print(f"Error with {name} engine: {str(e)}")
    
    # Display results
    for name, hotels in results.items():
        display(Markdown(f"### {name.upper()} Results"))
        
        for i, hotel in enumerate(hotels, 1):
            display(Markdown(f"#### {i}. {hotel.title}"))
            display(Markdown(f"**Score:** {hotel.score:.2f}"))
            display(Markdown(f"**URL:** {hotel.url}"))
            display(Markdown(f"**Snippet:** {hotel.snippet}"))
            
            if hotel.metadata:
                display(Markdown("**Metadata:**"))
                for key, value in hotel.metadata.items():
                    if value:  # Only show non-empty values
                        display(Markdown(f"- {key}: {value}"))
            
            # Display raw response if available
            if hotel.raw_response:
                display(Markdown("**Raw Response:**"))
                display(Markdown(f"```json\n{json.dumps(hotel.raw_response, indent=2)}\n```"))
            
            display(Markdown("---"))

## Example Searches

Let's try some example searches to see how each engine performs.

In [75]:
# Search for luxury beachfront hotels
search_hotels("luxury beachfront hotels with ocean view and spa facilities")

2025-05-06 01:08:41 - INFO - search took 0.11 seconds
2025-05-06 01:08:42 - ERROR - Error in search: DuckDuckGo search failed: https://html.duckduckgo.com/html 202 Ratelimit


Error with duckduckgo engine: DuckDuckGo search failed: https://html.duckduckgo.com/html 202 Ratelimit


2025-05-06 01:08:53 - INFO - search took 10.96 seconds
2025-05-06 01:08:54 - INFO - search took 0.54 seconds


### GENERIC Results

#### 1. The Palms Hotel & Spa

...

- number_of_reviews: 3368

- ranking: #2 of 235 hotels in Miami Beach

- phone: 13056975834

- website: https://www.standardhotels.com/miami/properties/miami-beach

---

In [76]:
# Search for family-friendly hotels
search_hotels("family-friendly hotels with swimming pools and activities for kids")

2025-05-06 01:08:54 - INFO - search took 0.11 seconds
2025-05-06 01:08:56 - ERROR - Error in search: DuckDuckGo search failed: https://html.duckduckgo.com/html 202 Ratelimit


Error with duckduckgo engine: DuckDuckGo search failed: https://html.duckduckgo.com/html 202 Ratelimit


2025-05-06 01:09:10 - INFO - search took 13.88 seconds
2025-05-06 01:09:10 - INFO - search took 0.56 seconds


### GENERIC Results

#### 1. Marriott Stanton South Beach

...

- number_of_reviews: 2933

- ranking: #107 of 235 hotels in Miami Beach

- phone: 13055749097

- website: https://www.sagamoresouthbeach.com/

---

## Try Your Own Search

Enter your own search query to find hotels that match your criteria.

In [77]:
# Your search query
query = "budget hotels near Miami Beach with free breakfast"
search_hotels(query)

2025-05-06 01:09:10 - INFO - search took 0.15 seconds
2025-05-06 01:09:12 - ERROR - Error in search: DuckDuckGo search failed: https://html.duckduckgo.com/html 202 Ratelimit


Error with duckduckgo engine: DuckDuckGo search failed: https://html.duckduckgo.com/html 202 Ratelimit


2025-05-06 01:09:29 - INFO - search took 17.02 seconds
2025-05-06 01:09:30 - INFO - search took 0.52 seconds


### GENERIC Results

#### 1. The Local House

...

- number_of_reviews: 782

- ranking: #46 of 235 hotels in Miami Beach

- phone: 13055385529

- website: https://localhouse.com

---